In [2]:
# Import libraries

import pandas as pd
from datetime import datetime, timedelta
import dateutil.parser as parser
import requests

In [3]:
# Read and clean CSV Sources

properties_cols = ['property_id','latitude','longitude','localidad', 'city', 'country']
convert_pro = lambda x: (x.replace('\xad','').title())  # function used to clean the locality column, Chí­A -> Chí­a
df_properties = pd.read_csv('sources\properties.csv', usecols = properties_cols, converters = {'localidad':convert_pro})

users_cols = ['property_id','user_id']
df_users = pd.read_csv('sources\\users.csv', usecols = users_cols)

visits_cols = ['scheduled_id','property_id','begin_date','status']
convert_vis = lambda x: (parser.parse(x) + timedelta(hours=2))  # function used to change the time UTC-3 to UTC-5 (Colombian time) in datetime columns
df_visits = pd.read_csv('sources\\visits.csv', usecols = visits_cols, converters = {'begin_date': convert_vis})
df_visits['visit_date'] = df_visits['begin_date'].apply(lambda x: datetime.strftime(x, "%Y-%m-%d"))
df_visits = df_visits.drop(columns='begin_date')


In [131]:
# Join CSV sources to create df_base and extend the visit's information

df_base = pd.merge(df_properties,df_visits, on='property_id', how='left')
df_base = pd.merge(df_base,df_users, on='property_id', how='left')
df_base['key_api'] = df_base['latitude'].map(str)+','+df_base['longitude'].map(str)+'/'+df_base['visit_date'].map(str)


In [108]:
# Get data from API

def rainy_day(conditions):
    rain_conditions = ['type_21','type_5','type_6','type_9','type_10','type_11','type_13',\
                       'type_14','type_22','type_23','type_24','type_25','type_26','type_32']
    conditions_list = [i.strip() for i in conditions.split(',')]
    
    for i in conditions_list:
        if i in rain_conditions:
            return True
    
    return False

def request(key):
    API_KEY = '2DAAC28RT2V8Q2SWYQVXV3DZF'
    endpoint = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{key}'
    query_params = {'unitGroup':'metric', 'key':API_KEY, 'include':'days','lang':'id'}
    
    response = requests.get(endpoint, params=query_params).json()
    temp = response['days'][0]['temp']
    wheather = rainy_day(response['days'][0]['conditions'])

    return [temp, wheather]


df_base_dd = df_base['key_api'].drop_duplicates().to_frame()
#df_base_dd['api_response'] = df_base_dd['key_api'].apply(lambda key: request(key))


In [128]:
#df_base_dd.to_csv('out_api.csv', index=False)

out_api_df = pd.read_csv('out_api.csv')
#df_base_dd = out_api_df
backup = df_base_dd
#df_base_dd


In [136]:
# DF with API's information

df_base_dd['temperature'] = df_base_dd['api_response'].apply(lambda x: x[0])
df_base_dd['rainy_day'] = df_base_dd['api_response'].apply(lambda x: x[1])
df_base_dd_data = df_base_dd.drop(columns='api_response')


In [139]:
# Create df_master with information to response challenge's questions 

df_master = pd.merge(df_base,df_base_dd_data, on='key_api', how='left')
df_master = df_master.drop(columns=['latitude','longitude','key_api'])
df_master

,property_id,localidad,city,country,scheduled_id,status,visit_date,user_id,temperature,rainy_day
0,1,Chía,Cundinamarca,Colombia,169548,Cancelled,2022-01-13,17,12.8,True
1,1,Chía,Cundinamarca,Colombia,184763,Cancelled,2022-01-26,17,14.9,True
2,1,Chía,Cundinamarca,Colombia,186092,Cancelled,2022-01-28,17,14.5,True
3,1,Chía,Cundinamarca,Colombia,182497,Cancelled,2022-01-23,17,15.0,True
4,1,Chía,Cundinamarca,Colombia,182396,Cancelled,2022-01-23,17,15.0,True
...,...,...,...,...,...,...,...,...,...,...
420,80,Rafael Uribe Uribe,Bogotá,Colombia,208045,Done,2022-02-27,80,14.0,True
421,80,Rafael Uribe Uribe,Bogotá,Colombia,208611,Cancelled,2022-02-28,80,13.4,True
422,80,Rafael Uribe Uribe,Bogotá,Colombia,207460,Done,2022-02-26,80,14.6,True
423,80,Rafael Uribe Uribe,Bogotá,Colombia,208040,Cancelled,2022-02-27,80,14.0,True


In [4]:
df_master = pd.read_csv('df_master.csv')
df_master


,property_id,localidad,city,country,scheduled_id,status,visit_date,user_id,temperature,rainy_day
0,1,Chía,Cundinamarca,Colombia,169548,Cancelled,2022-01-13,17,12.8,True
1,1,Chía,Cundinamarca,Colombia,184763,Cancelled,2022-01-26,17,14.9,True
2,1,Chía,Cundinamarca,Colombia,186092,Cancelled,2022-01-28,17,14.5,True
3,1,Chía,Cundinamarca,Colombia,182497,Cancelled,2022-01-23,17,15.0,True
4,1,Chía,Cundinamarca,Colombia,182396,Cancelled,2022-01-23,17,15.0,True
...,...,...,...,...,...,...,...,...,...,...
420,80,Rafael Uribe Uribe,Bogotá,Colombia,208045,Done,2022-02-27,80,14.0,True
421,80,Rafael Uribe Uribe,Bogotá,Colombia,208611,Cancelled,2022-02-28,80,13.4,True
422,80,Rafael Uribe Uribe,Bogotá,Colombia,207460,Done,2022-02-26,80,14.6,True
423,80,Rafael Uribe Uribe,Bogotá,Colombia,208040,Cancelled,2022-02-27,80,14.0,True


In [39]:
# Challenge's questions

print('1. ¿Cuántas visitas se realizaron en total?')
q1 = len(df_master.query('status == "Done"'))
q1_all = len(df_master)
print(f'Rta: Se realizaron {q1} visitas sobre un total de {q1_all} visitas programadas, es decir, se llevo a cabo el {round(q1*100/q1_all, 2)}% de las visitas programadas. \n')

print('2. ¿Cuál es el promedio de propiedades por usuario?')
q2 = (df_users.groupby('user_id')['property_id'].count()).mean()
q2_func = lambda x: "propiedades" if int(x)>1 else "propiedad"
print(f'Rta: Cada usuario tiene en promedio {int(q2)} {q2_func(q2)}.\n')

print('3. ¿Cuál era la temperatura promedio de todas las visitas que realizó el usuario con ID 2?')
q3 = df_master.query('user_id == 2 and status == "Done"')['temperature'].mean()
print(f'Rta: La temperatura promedio de las visitas realizadas en la propiedad del usuario con ID 2 fue de {round(q3,2)}°C.\n')

print('4. ¿Cuál es la temperatura promedio de las visitas para los días con lluvia?')
q4 = df_master.query('status == "Done" and rainy_day == True')['temperature'].mean()
print(f'Rta: La temperatura promedio de las visitas realizadas en días con lluvia fue de {round(q4,2)}°C.\n')

print('5. ¿Cuál es la temperatura promedio para las visitas realizadas en la localidad de Suba?')
q5 = df_master.query('localidad == "Suba" and status == "Done"')['temperature'].mean()
print(f'Rta: La temperatura promedio de las visitas realizadas en la localidad de Suba es de {round(q5,2)}°C.\n')


1. ¿Cuántas visitas se realizaron en total?
Rta: Se realizaron 139 visitas sobre un total de 425 visitas programadas, es decir, se llevo a cabo el 32.71% de las visitas programadas. 

2. ¿Cuál es el promedio de propiedades por usuario?
Rta: Cada usuario tiene en promedio 1 propiedad.

3. ¿Cuál era la temperatura promedio de todas las visitas que realizó el usuario con ID 2?
Rta: La temperatura promedio de las visitas realizadas en la propiedad del usuario con ID 2 fue de 14.6°C.

4. ¿Cuál es la temperatura promedio de las visitas para los días con lluvia?
Rta: La temperatura promedio de las visitas realizadas en días con lluvia fue de 14.2°C.

5. ¿Cuál es la temperatura promedio para las visitas realizadas en la localidad de Suba?
Rta: La temperatura promedio de las visitas realizadas en la localidad de Suba es de 14.12°C.



In [ ]:
# Additional analyzes

df_qa = (df_master.groupby('status')['property_id'].count()).mean()

df_master.query('user_id == 2 and status == "Done"')['temperature'].mean()



q1_cancelled = (q1_all - q1)
round((q1_cancelled*100/q1_all), 2)
